### Load Snowpark libraries

In [61]:
# Import required libraries
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import avg, sum, col,lit
from snowflake.snowpark.functions import udf, sproc, col
from snowflake.snowpark.types import IntegerType, FloatType, LongType, DoubleType, DecimalType,StringType, BooleanType, Variant
from snowflake.snowpark.types import PandasSeries, PandasDataFrame
from snowflake.snowpark import functions as fn

import sys ,json
import io
import logging
import pandas as pd

import joblib
import pandas as pd
import numpy as np
import json

from snowflake.snowpark import version
print (f"snowflake snowpark version is: {version.VERSION}")

snowflake snowpark version is: (0, 10, 0)


### Connect to Snowflake and establish session

In [62]:
snowflake_connection_cfg = open('cred.json')
snowflake_connection_cfg = snowflake_connection_cfg.read()
snowflake_connection_cfg = json.loads(snowflake_connection_cfg)

# Creating Snowpark Session
tc_session = Session.builder.configs(snowflake_connection_cfg).create()
print('Current Database:', tc_session.get_current_database())
print('Current Schema:', tc_session.get_current_schema())
print('Current Warehouse:', tc_session.get_current_warehouse())

Current Database: "BANK1_CRM_DB"
Current Schema: "PUBLIC"
Current Warehouse: "APP_WH"


### Create stage location for models

In [63]:
tc_session.sql("CREATE OR REPLACE STAGE stage_models").collect()

[Row(status='Stage area STAGE_MODELS successfully created.')]

In [64]:
tc_session.clear_packages()
tc_session.add_packages("snowflake-snowpark-python")
tc_session.add_packages("scikit-learn","pandas","numpy","joblib","cachetools")
tc_session.clear_imports()

### Define function to train test random split

In [65]:
def train_test_split(training_table: str, sample_size_n: int, features:list, Y:str, test_size:float, random_state:int):
    # Loading data into pandas dataframe

    # Define features and label
    X = training_table[features]
    Y = training_table[Y]

    # Splitting data into training and test
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=random_state)

    return X_train, X_test, y_train, y_test

### Define function to save trained model

In [66]:
def save_file(session, model, path, dest_filename):
    # logger.debug('#save_file: -- START--')
    input_stream = io.BytesIO()
    joblib.dump(model, input_stream)
    session._conn.upload_stream(input_stream, path, dest_filename)
    return "successfully created file: " + path

### Define Features required to train model

In [67]:
features=['SEPTAL_LENGTH','SEPTAL_WIDTH','PETAL_LENGTH','PETAL_WIDTH']

### Define Model pipeline for Imputer, Standard Scaler and Random Classifier Model

In [68]:
def build_rf_model(p_df: pd.DataFrame,ne,nj,cw, md):
    from sklearn.pipeline import Pipeline
    from sklearn.impute import SimpleImputer
    from sklearn.preprocessing import StandardScaler, OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.ensemble import RandomForestClassifier
    numeric_features = p_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_features = p_df.select_dtypes(include=['object']).columns.tolist()

    feature_names = numeric_features + categorical_features

    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler(with_mean=True,with_std=True))])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])

    model = Pipeline(steps=[
                ('preprocessor', preprocessor),
                ('classifier'
                    ,RandomForestClassifier(n_estimators=ne, n_jobs=-nj, class_weight=cw,max_depth=md)
                    # ,RandomForestClassifier(n_estimators=4, n_jobs=-1, class_weight='balanced_subsample',max_depth=20)
                    # ,RandomForestClassifier(maxBins=20,featureSubsetStrategy='onethird') need to find the equivalents
                    # of these maxBins and featureSubsetStrategy. For featureSubsetStrategy I do think it is the 
                    # classweight from sklearn based on the documentation. I also think maxBins could be the same as
                    # maxdepth.
                )
            ])

    return model


In [69]:
def build_dtree_model(p_df: pd.DataFrame,cw, md):
    from sklearn.pipeline import Pipeline
    from sklearn.impute import SimpleImputer
    from sklearn.preprocessing import StandardScaler, OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier

    numeric_features = p_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_features = p_df.select_dtypes(include=['object']).columns.tolist()

    feature_names = numeric_features + categorical_features

    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler(with_mean=True,with_std=True))])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])

    model = Pipeline(steps=[
                ('preprocessor', preprocessor),
                ('classifier'
                    ,DecisionTreeClassifier(class_weight=cw,max_depth=md)
                    # ,RandomForestClassifier(n_estimators=4, n_jobs=-1, class_weight='balanced_subsample',max_depth=20)
                    # ,RandomForestClassifier(maxBins=20,featureSubsetStrategy='onethird') need to find the equivalents
                    # of these maxBins and featureSubsetStrategy. For featureSubsetStrategy I do think it is the 
                    # classweight from sklearn based on the documentation. I also think maxBins could be the same as
                    # maxdepth.
                )
            ])

    return model

In [70]:
def get_classification_report(y_test, y_pred):
    from sklearn import metrics
    report = metrics.classification_report(y_test, y_pred, output_dict=True,target_names=['setosa', 'versicolor', 'virginica'])
    df_classification_report = pd.DataFrame(report).transpose()    
    return df_classification_report

In [71]:
def get_model_info(model_name, test_size, random_state,ne,nj,cw,max_depth):
    data = [[model_name,test_size,random_state,ne,nj,cw,max_depth]]  
    df_model_info = pd.DataFrame(data,columns=['model','test_size','random_state','n_estimator','n_jobs','class_weight','max_depth'])
    return df_model_info

### Train random forest classifier model

In [72]:
def train_rf_model(session: Session, training_table: str, sample_size_n: int, model_name: str,features:list, Y: str,test_size:float,random_state:int,ne:int,nj:int,cw:str, md:int) -> str:
    from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score
    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

    training_data = session.table(training_table).sample(n=sample_size_n).toPandas()

    Data_train, Data_test, y_train, y_test = train_test_split(training_data, sample_size_n, features, Y,test_size,random_state)
    from sklearn.ensemble import RandomForestClassifier 
    # Model building
    rf = build_rf_model(Data_train,ne,nj,cw, md)
    rf.fit(Data_train, y_train)

    model_dir = '@stage_models'
    model_fl = model_name+'.joblib'
    save_file(session, rf, model_dir ,model_fl)

    score = rf.score(Data_test, y_test)
    
    y_pred = rf.predict(Data_test)
    df_classification_report = get_classification_report(y_pred,y_test).reset_index().rename(columns={"index": "class"}).reset_index(drop=True)
    df_model_info = get_model_info(model_fl,test_size,random_state,ne,nj,cw,md)
    df_model_info=df_model_info.append([df_model_info]*5,ignore_index=True)
    # tc_session.create_dataframe(df_classification_report.join(df_model_info)).write.mode("overwrite").save_as_table("model_output")
    
    return df_classification_report.join(df_model_info)

In [73]:
def train_dtree_model(session: Session, training_table: str, sample_size_n: int, model_name: str,features:list, Y: str,test_size:float,random_state:int,cw:str, md:int) -> str:
    training_data = session.table(training_table).sample(n=sample_size_n).toPandas()

    Data_train, Data_test, y_train, y_test = train_test_split(training_data, sample_size_n, features, Y,test_size,random_state)
    # Model building
    dtree = build_dtree_model(Data_train,cw, md)
    dtree.fit(Data_train, y_train)

    model_dir = '@stage_models'
    model_fl = model_name+'.joblib'
    save_file(session, dtree, model_dir ,model_fl)
    
    y_pred = dtree.predict(Data_test)
    df_classification_report = get_classification_report(y_pred,y_test).reset_index().rename(columns={"index": "class"}).reset_index(drop=True)
    df_model_info = get_model_info(model_fl,test_size,random_state,None,None,cw,md)
    df_model_info=df_model_info.append([df_model_info]*5,ignore_index=True)
    # tc_session.create_dataframe(df_classification_report.join(df_model_info)).write.mode("overwrite").save_as_table("model_output")
    
    return df_classification_report.join(df_model_info)

### Define stored proc to register random forest classifier model

In [74]:
# Registering the function as a Stored Procedure
rf_sproc = tc_session.sproc.register(func=train_rf_model, # training function defined above
                                            name='train_rf_model', # training model name to be registered in snowlake
                                            is_permanent=True, # permanent stored proc
                                            replace=True, # replace if existing already
                                            stage_location='@stage_models', # save the model in stage location
                                            packages=['snowflake-snowpark-python','scikit-learn','joblib']) # import model libaries


In [75]:
# Registering the function as a Stored Procedure
dtree_sproc = tc_session.sproc.register(func=train_dtree_model, # training function defined above
                                            name='train_dtree_model', # training model name to be registered in snowlake
                                            is_permanent=True, # permanent stored proc
                                            replace=True, # replace if existing already
                                            stage_location='@stage_models', # save the model in stage location
                                            packages=['snowflake-snowpark-python','scikit-learn','joblib']) # import model libaries


In [78]:
print ("random forest classifier report")
print (rf_sproc('iris_dataset',100, 'rf_iris_model_ts10_md15',features,'LABEL',0.1,43,4,1,'balanced_subsample', 15))

random forest classifier report
          class  precision  recall  ...  n_jobs        class_weight max_depth
0        setosa        1.0     1.0  ...       1  balanced_subsample        15
1    versicolor        1.0     1.0  ...       1  balanced_subsample        15
2     virginica        1.0     1.0  ...       1  balanced_subsample        15
3      accuracy        1.0     1.0  ...       1  balanced_subsample        15
4     macro avg        1.0     1.0  ...       1  balanced_subsample        15
5  weighted avg        1.0     1.0  ...       1  balanced_subsample        15

[6 rows x 12 columns]


In [99]:
table_name = 'iris_dataset'
sample = 100,
test_size = 0.1
max_depth = 15
model_name = 'rf_iris_model_'+'ts'+str(test_size)+'_'+'md'+str(max_depth)
random_state = 43,
n_estimator = 4
n_jobs = 1,
class_weight = 'balanced_subsample'
print ("random forest classifier report")
print (rf_sproc(table_name
                ,100
                , model_name
                ,features
                ,'LABEL'
                ,test_size
                ,43
                ,n_estimator
                ,1
                ,class_weight
                , max_depth))

random forest classifier report
          class  precision  recall  ...  n_jobs        class_weight max_depth
0        setosa        1.0     1.0  ...       1  balanced_subsample        15
1    versicolor        1.0     1.0  ...       1  balanced_subsample        15
2     virginica        1.0     1.0  ...       1  balanced_subsample        15
3      accuracy        1.0     1.0  ...       1  balanced_subsample        15
4     macro avg        1.0     1.0  ...       1  balanced_subsample        15
5  weighted avg        1.0     1.0  ...       1  balanced_subsample        15

[6 rows x 12 columns]


In [100]:
table_name = 'iris_dataset'
sample = 100,
test_size = 0.25
max_depth = 20
model_name = 'rf_iris_model_'+'ts'+str(test_size)+'_'+'md'+str(max_depth)
random_state = 43,
n_estimator = 4
n_jobs = 1,
class_weight = 'balanced_subsample'
print ("random forest classifier report")
print (rf_sproc(table_name
                ,100
                , model_name
                ,features
                ,'LABEL'
                ,test_size
                ,43
                ,n_estimator
                ,1
                ,class_weight
                , max_depth))

random forest classifier report
          class  precision    recall  ...  n_jobs        class_weight max_depth
0        setosa   1.000000  1.000000  ...       1  balanced_subsample        20
1    versicolor   1.000000  0.888889  ...       1  balanced_subsample        20
2     virginica   0.900000  1.000000  ...       1  balanced_subsample        20
3      accuracy   0.960000  0.960000  ...       1  balanced_subsample        20
4     macro avg   0.966667  0.962963  ...       1  balanced_subsample        20
5  weighted avg   0.964000  0.960000  ...       1  balanced_subsample        20

[6 rows x 12 columns]


In [101]:
table_name = 'iris_dataset'
sample = 100,
test_size = 0.25
max_depth = 25
model_name = 'rf_iris_model_'+'ts'+str(test_size)+'_'+'md'+str(max_depth)
random_state = 43,
n_estimator = 4
n_jobs = 1,
class_weight = 'balanced_subsample'
print ("random forest classifier report")
print (rf_sproc(table_name
                ,100
                , model_name
                ,features
                ,'LABEL'
                ,test_size
                ,43
                ,n_estimator
                ,1
                ,class_weight
                , max_depth))

random forest classifier report
          class  precision  recall  ...  n_jobs        class_weight max_depth
0        setosa        1.0     1.0  ...       1  balanced_subsample        25
1    versicolor        1.0     1.0  ...       1  balanced_subsample        25
2     virginica        1.0     1.0  ...       1  balanced_subsample        25
3      accuracy        1.0     1.0  ...       1  balanced_subsample        25
4     macro avg        1.0     1.0  ...       1  balanced_subsample        25
5  weighted avg        1.0     1.0  ...       1  balanced_subsample        25

[6 rows x 12 columns]


In [104]:
table_name = 'iris_dataset'
sample = 100,
test_size = 0.25
max_depth = 20
model_name = 'dtree_iris_model_'+'ts'+str(test_size)+'_'+'md'+str(max_depth)
random_state = 43,
class_weight = 'balanced'
print ("decision tree classifier report")
print (dtree_sproc(table_name
                ,100
                , model_name
                ,features
                ,'LABEL'
                ,test_size
                ,43
                ,class_weight
                , max_depth))

decision tree classifier report
          class  precision    recall  ...  n_jobs  class_weight max_depth
0        setosa   1.000000  1.000000  ...    None      balanced        20
1    versicolor   1.000000  0.833333  ...    None      balanced        20
2     virginica   0.900000  1.000000  ...    None      balanced        20
3      accuracy   0.960000  0.960000  ...    None      balanced        20
4     macro avg   0.966667  0.944444  ...    None      balanced        20
5  weighted avg   0.964000  0.960000  ...    None      balanced        20

[6 rows x 12 columns]


In [105]:
table_name = 'iris_dataset'
sample = 100,
test_size = 0.25
max_depth = 25
model_name = 'dtree_iris_model_'+'ts'+str(test_size)+'_'+'md'+str(max_depth)
random_state = 43,
class_weight = 'balanced'
print ("decision tree classifier report")
print (dtree_sproc(table_name
                ,100
                , model_name
                ,features
                ,'LABEL'
                ,test_size
                ,43
                ,class_weight
                , max_depth))

decision tree classifier report
          class  precision    recall  ...  n_jobs  class_weight max_depth
0        setosa   1.000000  1.000000  ...    None      balanced        25
1    versicolor   1.000000  0.636364  ...    None      balanced        25
2     virginica   0.636364  1.000000  ...    None      balanced        25
3      accuracy   0.840000  0.840000  ...    None      balanced        25
4     macro avg   0.878788  0.878788  ...    None      balanced        25
5  weighted avg   0.898182  0.840000  ...    None      balanced        25

[6 rows x 12 columns]


In [106]:
table_name = 'iris_dataset'
sample = 100,
test_size = 0.25
max_depth = 30
model_name = 'dtree_iris_model_'+'ts'+str(test_size)+'_'+'md'+str(max_depth)
random_state = 43,
class_weight = 'balanced'
print ("decision tree classifier report")
print (dtree_sproc(table_name
                ,100
                , model_name
                ,features
                ,'LABEL'
                ,test_size
                ,43
                ,class_weight
                , max_depth))

decision tree classifier report
          class  precision    recall  ...  n_jobs  class_weight max_depth
0        setosa   1.000000  1.000000  ...    None      balanced        30
1    versicolor   1.000000  0.800000  ...    None      balanced        30
2     virginica   0.833333  1.000000  ...    None      balanced        30
3      accuracy   0.920000  0.920000  ...    None      balanced        30
4     macro avg   0.944444  0.933333  ...    None      balanced        30
5  weighted avg   0.933333  0.920000  ...    None      balanced        30

[6 rows x 12 columns]


### Check if the classifier models are saved in stage location.
### Remember if the same model name was used for all the iterations, then only the last trained model will be saved
### for the model name.

In [107]:
tc_session.sql("list @stage_models").collect()

[Row(name='stage_models/dtree_iris_model_ts0.25_md20.joblib', size=4464, md5='7be5315171da46a1dcb589f8880ee08b', last_modified='Mon, 28 Nov 2022 03:52:17 GMT'),
 Row(name='stage_models/dtree_iris_model_ts0.25_md25.joblib', size=3984, md5='a289f71f483766020b85a699b0199795', last_modified='Mon, 28 Nov 2022 03:52:39 GMT'),
 Row(name='stage_models/dtree_iris_model_ts0.25_md30.joblib', size=4464, md5='e1af13d86322e437c27e06374afa579e', last_modified='Mon, 28 Nov 2022 03:52:53 GMT'),
 Row(name='stage_models/rf_iris_model_ts0.1_md15.joblib', size=7856, md5='d61cde6c66717487a40a158adb3f4399', last_modified='Mon, 28 Nov 2022 03:49:31 GMT'),
 Row(name='stage_models/rf_iris_model_ts0.25_md20.joblib', size=9296, md5='dcf71d608370c8158081b3c908be8065', last_modified='Mon, 28 Nov 2022 03:50:03 GMT'),
 Row(name='stage_models/rf_iris_model_ts0.25_md25.joblib', size=8656, md5='2fa5d0e4663ab41e2d683192e01a5053', last_modified='Mon, 28 Nov 2022 03:50:30 GMT'),
 Row(name='stage_models/rf_iris_model_ts10_m

In [108]:
tc_session.close()
print('Finished!!!')

Finished!!!
